In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyqpanda as pq
import pyvqnet as pv
import random

In [2]:
pic_size = 32
output_num = 10
input_channels = 3
output_channels = 1
quantum_number = 4
stride = (4, 4)
pic_w = 32
pic_h = 32
output_num = 10

epoch = 3
batch = 100


class Model(pv.nn.Module):
    def __init__(self):
        super().__init__()
        self.vq = pv.qnn.qcnn.QConv(
            input_channels, output_channels, quantum_number, stride
        )
        self.li = pv.nn.Linear(
            output_channels * (pic_w // stride[0]) * (pic_h // stride[1]), output_num
        )

    def forward(self, x):
        x = self.vq(x)
        x = pv.tensor.flatten(x, 1)
        x = self.li(x)
        x = pv.tensor.log_softmax(x)
        return x

In [3]:
def unpickle(file):
    import pickle

    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict

In [4]:
def dataloader(file):
    data = unpickle(file)
    X = data[b"data"]
    Y = data[b"labels"]
    N = X.shape[0]
    L = [i for i in range(N)]
    random.shuffle(L)
    now = 0
    pic_tot = pic_w * pic_h
    while now < N:
        n = min(N - now, batch)
        x = np.zeros([n, input_channels, pic_w, pic_h])
        y = np.zeros([n], dtype="int64")
        for i in range(n):
            for c in range(input_channels):
                x[i, c] = (
                    X[L[now]][pic_tot * c : pic_tot * (c + 1)].reshape((pic_w, pic_h))
                    / 255
                )
            y[i] = Y[L[now]]
            now += 1
        yield x, y

In [5]:
# y=1
# for x,y in dataloader("data/cifar-10-batches-py/data_batch_1"):
#     print(x.shape,y.shape)
#     print(x)
#     break

In [6]:
model = Model()
model.train()
optimizer = pv.optim.SGD(model.parameters())
cceloss = pv.nn.loss.NLL_Loss()

for e in range(epoch):
    full_loss = 0
    n_loss = 0
    n_eval = 0
    correct = 0
    for i, (x, y) in enumerate(dataloader("data/cifar-10-batches-py/data_batch_1")):
        print("epoch", e, "\tbatch", i, end = " ")
        
        optimizer.zero_grad()
        output = model(x)
        loss = cceloss(y, output)

        loss.backward()
        optimizer._step()

        full_loss += loss.item()
        n_loss += batch

        np_output = np.array(output.data, copy=False)
        mask = np_output.argmax(1) == y
        correct += sum(mask)

        print("\tloss", loss, "\tcorrect", correct)
    print(f"Train Accuracy: {correct/n_loss}%")
    print(f"Epoch: {epoch}, Loss: {full_loss / n_loss}")

epoch 0 	batch 0 	loss 2.3808026 	correct 8
epoch 0 	batch 1 	loss 2.3498964 	correct 19
epoch 0 	batch 2 	loss 2.3346283 	correct 29
epoch 0 	batch 3 	loss 2.340718 	correct 39
epoch 0 	batch 4 	loss 2.3235254 	correct 48
epoch 0 	batch 5 	loss 2.344951 	correct 56
epoch 0 	batch 6 	loss 2.3369017 	correct 64
epoch 0 	batch 7 	loss 2.3292806 	correct 80
epoch 0 	batch 8 	loss 2.3096075 	correct 94
epoch 0 	batch 9 	loss 2.3946567 	correct 103
epoch 0 	batch 10 	loss 2.3420844 	correct 114
epoch 0 	batch 11 	loss 2.427244 	correct 119
epoch 0 	batch 12 	loss 2.3467836 	correct 127
epoch 0 	batch 13 	loss 2.3510702 	correct 136
epoch 0 	batch 14 	loss 2.3678482 	correct 146
epoch 0 	batch 15 	loss 2.3038132 	correct 157
epoch 0 	batch 16 	loss 2.4310868 	correct 165
epoch 0 	batch 17 	loss 2.3716319 	correct 174
epoch 0 	batch 18 	loss 2.34245 	correct 185
epoch 0 	batch 19 	loss 2.4065182 	correct 200
epoch 0 	batch 20 	loss 2.3085928 	correct 211
epoch 0 	batch 21 	loss 2.32447 	corre

KeyboardInterrupt: 

In [ ]:
pv.utils.storage.save_parameters(model.state_dict(), "train2.model")